In [ ]:
import ast
import json
import os
import pickle
import warnings
from collections import Counter
from pathlib import Path
from random import randint, random

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from google.colab import drive
from scipy.sparse import coo_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import euclidean_distances as ED
from tensorflow import keras
from tqdm.notebook import tqdm

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Считаем препроцессные данные

In [ ]:
!unzip processed_data.zip -x

Archive:  processed_data.zip
  inflating: users_processed_kion.csv  
  inflating: interactions_processed_kion.csv  
  inflating: items_processed_kion.csv  


In [ ]:
interactions_df = pd.read_csv('/content/interactions_processed_kion.csv')
users_df = pd.read_csv('/content/users_processed_kion.csv')
items_df = pd.read_csv('/content/items_processed_kion.csv')

In [ ]:
users_df.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,M,True
1,962099,age_18_24,income_20_40,M,False
2,1047345,age_45_54,income_40_60,F,False
3,721985,age_45_54,income_20_40,F,False
4,704055,age_35_44,income_60_90,F,False


In [ ]:
items_df = items_df.rename(columns={"id": "item_id"})

In [ ]:
items_df.head()

,item_id,content_type,title,title_orig,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,release_year_cat
0,10711,film,поговори с ней,Hable con ella,"драмы, зарубежные, детективы, мелодрамы",испания,False,16.0,unknown,педро альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...",2000-2010
1,2508,film,голые перцы,Search Party,"зарубежные, приключения, комедии",сша,False,16.0,unknown,скот армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...",2010-2020
2,10716,film,тактическая сила,Tactical Force,"криминал, зарубежные, триллеры, боевики, комедии",канада,False,16.0,unknown,адам п. калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...",2010-2020
3,7868,film,45 лет,45 Years,"драмы, зарубежные, мелодрамы",великобритания,False,16.0,unknown,эндрю хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю...",2010-2020
4,16268,film,все решает мгновение,None,"драмы, спорт, советские, мелодрамы",ссср,False,12.0,ленфильм,виктор садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...",1970-1980


In [ ]:
interactions_df.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72
1,699317,1659,2021-05-29,8317,100
2,656683,7107,2021-05-09,10,0
3,864613,7638,2021-07-05,14483,100
4,964868,9506,2021-04-30,6725,100


## Готовим фичи пользователей

Посмотрим, какие фичи в датасете фильмов являются категориальными и закодируем их с помощью one-hot encoding.

In [ ]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]
# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами
# из датафрейма с взаимодействиями
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
    # получаем датафрейм с one-hot encoding для каждой категориальной фичи
    ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
    # конкатенируем ohe-hot датафрейм с датафреймом,
    # который мы получили на предыдущем шаге
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()

,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,age_age_unknown,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,income_income_unknown,sex_F,sex_M,sex_sex_unknown,kids_flg_False,kids_flg_True
0,973171,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
1,962099,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
2,1047345,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0
3,721985,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
4,704055,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0


## Готовим фичи айтемов

Кодируем их точно так же - one-hot'ом.

In [ ]:
items_df.head()

,item_id,content_type,title,title_orig,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,release_year_cat
0,10711,film,поговори с ней,Hable con ella,"драмы, зарубежные, детективы, мелодрамы",испания,False,16.0,unknown,педро альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...",2000-2010
1,2508,film,голые перцы,Search Party,"зарубежные, приключения, комедии",сша,False,16.0,unknown,скот армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...",2010-2020
2,10716,film,тактическая сила,Tactical Force,"криминал, зарубежные, триллеры, боевики, комедии",канада,False,16.0,unknown,адам п. калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...",2010-2020
3,7868,film,45 лет,45 Years,"драмы, зарубежные, мелодрамы",великобритания,False,16.0,unknown,эндрю хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю...",2010-2020
4,16268,film,все решает мгновение,None,"драмы, спорт, советские, мелодрамы",ссср,False,12.0,ленфильм,виктор садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...",1970-1980


In [ ]:
item_cat_feats = ["content_type",  "release_year_cat",
    "for_kids",  "age_rating",
    "studios", "countries", "directors"]

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
    ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1)

items_ohe_df.head()

,item_id,content_type_film,content_type_series,release_year_cat_1920-1930,release_year_cat_1930-1940,release_year_cat_1940-1950,release_year_cat_1950-1960,release_year_cat_1960-1970,release_year_cat_1970-1980,release_year_cat_1980-1990,...,directors_ярив хоровиц,directors_ярон зильберман,directors_ярополк лапшин,directors_ярослав лупий,"directors_ярроу чейни, скотт моужер",directors_ясина сезар,directors_ясуоми умэцу,"directors_ёдзи фукуяма, ацуко фукусима, николас де креси, синъитиро ватанабэ, сёдзи кавамори",directors_ёлкин туйчиев,directors_ён сан-хо
0,10711,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2508,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10716,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7868,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16268,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


Добавим популярность как столбец:

In [ ]:
popular_items = (
    interactions_df.groupby("item_id")
    .agg({"user_id": "count"})
    .rename(columns={"user_id": "popularity"})
)

In [ ]:
popular_items.sort_values(by='popularity', ascending=False)

,popularity
item_id,
10440,202457
15297,193123
9728,132865
13865,122119
4151,91167
...,...
9098,1
3159,1
13219,1


In [ ]:
items_df = items_df.merge(popular_items, on="item_id")

In [ ]:
items_df.head(3)

,item_id,content_type,title,title_orig,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,release_year_cat,popularity
0,10711,film,поговори с ней,Hable con ella,"драмы, зарубежные, детективы, мелодрамы",испания,False,16.0,unknown,педро альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...",2000-2010,5
1,2508,film,голые перцы,Search Party,"зарубежные, приключения, комедии",сша,False,16.0,unknown,скот армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...",2010-2020,9
2,10716,film,тактическая сила,Tactical Force,"криминал, зарубежные, триллеры, боевики, комедии",канада,False,16.0,unknown,адам п. калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...",2010-2020,6


## Сделаем матрицу взаимодействий

В датасете взаимодействий есть непопулярные фильмы и малоактивные пользователи. Кроме того, в таблице взаимодействий есть события с низким качеством взаимодействия - когда юзер начал смотреть фильм, но вскоре после начала просмотра выключил.

Отфильтруем такие события*, малоактивных юзеров и непопулярные фильмы.


In [ ]:
print(f"N users before: {interactions_df.user_id.nunique()}")
print(f"N items before: {interactions_df.item_id.nunique()}\n")

# отфильтруем все события взаимодействий, в которых пользователь посмотрел
# фильм менее чем на 10 процентов
interactions_df = interactions_df[interactions_df.watched_pct > 10]

# соберем всех пользователей, которые посмотрели
# больше 10 фильмов (можете выбрать другой порог)
valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)

# и соберем все фильмы, которые посмотрели больше 10 пользователей
valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

# отбросим непопулярные фильмы и неактивных юзеров
interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 962179
N items before: 15706

N users after: 79515
N items after: 6901


После фильтрации может получиться так, что некоторые айтемы/юзеры есть в датасете взаимодействий, но при этом они отсутствуют в датасетах айтемов/юзеров или наоборот. Поэтому найдем id айтемов и id юзеров, которые есть во всех датасетах и оставим только их.

In [ ]:
common_users = set(interactions_df.user_id.unique()).intersection(
    set(users_ohe_df.user_id.unique())
)
common_items = set(interactions_df.item_id.unique()).intersection(
    set(items_ohe_df.item_id.unique())
)

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

65974
6901


Соберем взаимодействия в матрицу user*item так, чтобы в строках этой матрицы были user_id, в столбцах - item_id, а на пересечениях строк и столбцов - единица, если пользователь взаимодействовал с айтемом и ноль, если нет.

Такую матрицу удобно собирать в numpy array, однако нужно помнить, что numpy array индексируется порядковыми индексами, а нам же удобнее использовать item_id и user_id.

Создадим некие внутренние индексы для user_id и item_id - uid и iid. Для этого просто соберем все user_id и item_id и пронумеруем их по порядку.

In [ ]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


,user_id,item_id,last_watch_dt,total_dur,watched_pct,uid,iid
0,176549,9506,2021-05-11,4250,72,10616,3944
1,699317,1659,2021-05-29,8317,100,42131,675
6,1016458,354,2021-08-14,1672,25,61024,139
7,884009,693,2021-08-04,703,14,53150,279
14,5324,8437,2021-04-18,6598,92,310,3485


Отнормируем матрицу взаимодействий

In [ ]:
interactions_vec = np.zeros((interactions_df.uid.nunique(), interactions_df.iid.nunique()))

# Сделаю не просто 1/0 для взаимодействий, а некий вес в зависимости от процента просмотренного
for user_id, item_id, prcnt in zip(interactions_df.uid, interactions_df.iid, interactions_df.watched_pct):
    interactions_vec[user_id, item_id] = interactions_vec[user_id, item_id] + prcnt/100


res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]

In [ ]:
print(interactions_df.item_id.nunique())
print(items_ohe_df.item_id.nunique())
print(interactions_df.user_id.nunique())
print(users_ohe_df.user_id.nunique())

print(set(items_ohe_df.item_id.unique()) - set(interactions_df.item_id.unique()))

6897
6901
65974
65974
{11805, 9788, 11501, 1734}


Для того, чтобы можно было удобно превратить iid/uid в item_id/user_id и наоборот соберем словари

{iid: item_id}, {uid: user_id} и {item_id: iid}, {user_id: uid}.

In [ ]:
iid_to_item_id = (
    interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
)
item_id_to_iid = (
    interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]
)

uid_to_user_id = (
    interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
)
user_id_to_uid = (
    interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]
)

И проиндексируем датасеты users_ohe_df и items_ohe_df по внутренним айди:

In [ ]:
items_ohe_df

,item_id,content_type_film,content_type_series,release_year_cat_1920-1930,release_year_cat_1930-1940,release_year_cat_1940-1950,release_year_cat_1950-1960,release_year_cat_1960-1970,release_year_cat_1970-1980,release_year_cat_1980-1990,...,directors_ярив хоровиц,directors_ярон зильберман,directors_ярополк лапшин,directors_ярослав лупий,"directors_ярроу чейни, скотт моужер",directors_ясина сезар,directors_ясуоми умэцу,"directors_ёдзи фукуяма, ацуко фукусима, николас де креси, синъитиро ватанабэ, сёдзи кавамори",directors_ёлкин туйчиев,directors_ён сан-хо
8,9853,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,3526,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,15076,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,2904,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,2635,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15957,15610,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15958,6443,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15959,2367,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15960,10632,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
a = [11805, 11501, 9788, 1734]

In [ ]:
items_ohe_df = items_ohe_df[~items_ohe_df["item_id"].isin(a)]

In [ ]:
items_ohe_df["iid"] = items_ohe_df["item_id"].apply(lambda x: item_id_to_iid[x])
items_ohe_df = items_ohe_df.set_index("iid")

users_ohe_df["uid"] = users_ohe_df["user_id"].apply(lambda x: user_id_to_uid[x])
users_ohe_df = users_ohe_df.set_index("uid")

In [ ]:
def triplet_loss(y_true, y_pred, n_dims=128, alpha=0.4):
    # будем ожидать, что на вход функции прилетит три сконкатенированных
    # вектора - вектор юзера и два вектора айтема
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims : n_dims * 2]
    negative = y_pred[:, n_dims * 2 : n_dims * 3]

    # считаем расстояния от вектора юзера до вектора хорошего айтема
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    # и до плохого
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    # считаем лосс
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0)  # возвращаем ноль, если лосс отрицательный

    return loss

## Выделим текстовые признаки при помощи CountVectorizer

In [ ]:
vectorizer = CountVectorizer(tokenizer=nltk.word_tokenize)
tf_idf = TfidfTransformer()
selector = SelectPercentile(chi2, percentile=1)

In [ ]:
items_df["description"].head()

0    Мелодрама легендарного Педро Альмодовара «Пого...
1    Уморительная современная комедия на популярную...
2    Профессиональный рестлер Стив Остин («Все или ...
3    Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...
4    Расчетливая чаровница из советского кинохита «...
Name: description, dtype: object

In [ ]:
bow = vectorizer.fit_transform(items_df["description"])
tf_idf = tf_idf.fit_transform(bow)

In [ ]:
main_features = selector.fit_transform(tf_idf, items_df["popularity"])
text_features = pd.DataFrame(main_features.toarray(), columns=selector.get_feature_names_out())

In [ ]:
text_features["item_id"] = items_df["item_id"]
items_ohe_df = items_ohe_df.merge(text_features, on="item_id", how="left")
items_ohe_df = items_ohe_df.fillna(0)

## Генератор и семплирование

Сделаем простой генератор. Он будет брать рандромного юзера, и два разных айтема - хороший пример и плохой:
- хорошим примером будет тот айтем, который был взят из датасета взаимодействий в соответствии с распределением просмотренных айтемов для этого юзера;
- а плохим айтемом будет просто любой другой _случайный айтем_*


In [ ]:
def generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            # берем рандомный uid
            uid_i = randint(0, interactions.shape[0] - 1)
            # id хорошего айтема
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i])
            # id плохого айтема
            neg_i = np.random.choice(range(interactions.shape[1]))
            # фичи юзера
            uid_meta.append(users.iloc[uid_i])
            # вектор айтемов, с которыми юзер взаимодействовал
            uid_interaction.append(interactions_vec[uid_i])
            # фичи хорошего айтема
            pos.append(items.iloc[pos_i])
            # фичи плохого айтема
            neg.append(items.iloc[neg_i])

        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [
            np.array(uid_meta),
            np.array(uid_interaction),
        ]

In [ ]:
# инициализируем генератор
gen = generator(
    items=items_ohe_df.drop(["item_id"], axis=1),
    users=users_ohe_df.drop(["user_id"], axis=1),
    interactions=interactions_vec,
)

ret = next(gen)


print(f"вектор фичей юзера: {ret[0][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[0][1].shape}")
print(f"вектор 'хорошего' айтема: {ret[0][2].shape}")
print(f"вектор 'плохого' айтема: {ret[0][3].shape}")
print()
print(f"вектор фичей юзера: {ret[1][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[1][1].shape}")

вектор фичей юзера: (1024, 19)
вектор взаимодействий юзера с айтемами: (1024, 6897)
вектор 'хорошего' айтема: (1024, 9836)
вектор 'плохого' айтема: (1024, 9836)

вектор фичей юзера: (1024, 19)
вектор взаимодействий юзера с айтемами: (1024, 6897)


## Модель

In [ ]:
N_FACTORS = 128

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = (items_ohe_df.drop(["item_id"], axis=1).shape[1],)
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1],)

USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1],)

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 128
ITEM_MODEL_SHAPE: (9836,)
USER_META_MODEL_SHAPE: (19,)
USER_INTERACTION_MODEL_SHAPE: (6897,)


In [ ]:
def item_model(n_factors=N_FACTORS):
    # входной слой
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

    # полносвязный слой
    layer_1 = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(inp)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2 = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(layer_1)

    add = keras.layers.Add()([layer_1, layer_2])

    # выходной слой
    out = keras.layers.Dense(
        N_FACTORS,
        activation="linear",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(add)

    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    # входной слой для вектора фичей юзера (из users_ohe_df)
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    # входной слой для вектора просмотров (из iteractions_vec)
    inp_interaction = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

    # полносвязный слой
    layer_1_meta = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(inp_meta)

    layer_1_interaction = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(inp_interaction)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2_meta = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(layer_1_meta)

    add = keras.layers.Add()([layer_1_meta, layer_2_meta])

    # конкатенируем вектор фичей с вектором просмотров
    concat_meta_interaction = keras.layers.Concatenate()([add, layer_1_interaction])

    # выходной слой
    out = keras.layers.Dense(
        N_FACTORS,
        activation="linear",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(concat_meta_interaction)

    return keras.models.Model([inp_meta, inp_interaction], out)


# инициализируем модели юзера и айтема
i2v = item_model()
u2v = user_model()

# вход для вектора фичей юзера (из users_ohe_df)
ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
# вход для вектора просмотра юзера (из interactions_vec)
ancor_interaction_in = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

# вход для вектора "хорошего" айтема
pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
# вход для вектора "плохого" айтема
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

# получаем вектор юзера
ancor = u2v([ancor_meta_in, ancor_interaction_in])
# получаем вектор "хорошего" айтема
pos = i2v(pos_in)
# получаем вектор "плохого" айтема
neg = i2v(neg_in)

# конкатенируем полученные векторы
res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

# собираем модель
model = keras.models.Model([ancor_meta_in, ancor_interaction_in, pos_in, neg_in], res)

In [ ]:
model_name = "recsys_resnet_linear"

# логируем процесс обучения в тензорборд
t_board = keras.callbacks.TensorBoard(log_dir=f"runs/{model_name}")

# уменьшаем learning_rate, если лосс долго не уменьшается (в течение двух эпох)
decay = keras.callbacks.ReduceLROnPlateau(monitor="loss", patience=2, factor=0.8, verbose=1)

# сохраняем модель после каждой эпохи, если лосс уменьшился
check = keras.callbacks.ModelCheckpoint(
    filepath=model_name + "/epoch{epoch}-{loss:.2f}.h5", monitor="loss"
)

In [ ]:
# компилируем модель, используем оптимайзер Adam и triplet loss
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=triplet_loss, optimizer=opt)

In [ ]:
# модель айтема
item_model().summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 9836)]               0         []                            
                                                                                                  
 dense_7 (Dense)             (None, 128)                  1259008   ['input_8[0][0]']             
                                                                                                  
 dense_8 (Dense)             (None, 128)                  16384     ['dense_7[0][0]']             
                                                                                                  
 add_2 (Add)                 (None, 128)                  0         ['dense_7[0][0]',             
                                                                     'dense_8[0][0]']       

In [ ]:
# модель юзера
user_model().summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 19)]                 0         []                            
                                                                                                  
 dense_10 (Dense)            (None, 128)                  2432      ['input_9[0][0]']             
                                                                                                  
 dense_12 (Dense)            (None, 128)                  16384     ['dense_10[0][0]']            
                                                                                                  
 input_10 (InputLayer)       [(None, 6897)]               0         []                            
                                                                                            

In [ ]:
# общая модель
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 19)]                 0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 6897)]               0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 9836)]               0         []                            
                                                                                                  
 input_7 (InputLayer)        [(None, 9836)]               0         []                            
                                                                                            

# Обучение

In [ ]:
# начинаем обучение, не забывая дропнуть столбцы item_id и user_id
# из датафреймов при инициализации генератора.

# batch_size можно (и лучше) поставить побольше, если вы не органичены в ресурсах

model.fit(
    generator(
        items=items_ohe_df.drop(["item_id"], axis=1),
        users=users_ohe_df.drop(["user_id"], axis=1),
        interactions=interactions_vec,
        batch_size=64,
    ),
    steps_per_epoch=100,
    epochs=30,
    initial_epoch=0,
    callbacks=[decay, t_board, check],
)

Epoch 1/30
100/100 [==============================] - 99s 942ms/step - loss: 0.3054 - lr: 0.0010
Epoch 2/30
100/100 [==============================] - 98s 982ms/step - loss: 0.2203 - lr: 0.0010
Epoch 3/30
100/100 [==============================] - 92s 927ms/step - loss: 0.1980 - lr: 0.0010
Epoch 4/30
100/100 [==============================] - 93s 943ms/step - loss: 0.1861 - lr: 0.0010
Epoch 5/30
100/100 [==============================] - 91s 920ms/step - loss: 0.1770 - lr: 0.0010
Epoch 6/30
100/100 [==============================] - 92s 926ms/step - loss: 0.1746 - lr: 0.0010
Epoch 7/30
100/100 [==============================] - 91s 916ms/step - loss: 0.1579 - lr: 0.0010
Epoch 8/30
100/100 [==============================] - 94s 935ms/step - loss: 0.1607 - lr: 0.0010
Epoch 9/30
100/100 [==============================] - ETA: 0s - loss: 0.1602
Epoch 9: ReduceLROnPlateau reducing learning rate to 0.000800000037997961.
100/100 [==============================] - 94s 946ms/step - loss: 0.1602

### Пример

In [ ]:
# берем рандомного юзера
rand_uid = np.random.choice(list(users_ohe_df.index))

# получаем фичи юзера и вектор его просмотров айтемов
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[rand_uid]
user_interaction_vec = interactions_vec[rand_uid]

# берем рандомный айтем
rand_iid = np.random.choice(list(items_ohe_df.index))
# получаем фичи айтема
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[rand_iid]

# получаем вектор юзера
user_vec = u2v.predict(
    [np.array(user_meta_feats).reshape(1, -1), np.array(user_interaction_vec).reshape(1, -1)]
)

# и вектор айтема
item_vec = i2v.predict(np.array(item_feats).reshape(1, -1))

# считаем расстояние между вектором юзера и вектором айтема
ED(user_vec, item_vec)

1/1 [==============================] - 0s 59ms/step


array([[1.414828]], dtype=float32)

# Получим рекомендации

In [ ]:
# получаем фичи всех айтемов
items_feats = items_ohe_df.drop(["item_id"], axis=1).to_numpy()
# получаем векторы всех айтемов
items_vecs = i2v.predict(items_feats)

# считаем расстояния
dists = ED(user_vec, items_vecs)

216/216 [==============================] - 0s 2ms/step


Обернем в функцию, чтобы получать рекомендации на лету

In [ ]:
def recommend_to_user(user_id, items_vecs, topn=10):
    uid = user_id_to_uid[user_id]
    user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[uid]
    user_interaction_vec = interactions_vec[uid]
    user_vec = u2v.predict(
        [np.array(user_meta_feats).reshape(1, -1), np.array(user_interaction_vec).reshape(1, -1)],
        verbose=False,
    )
    dists = ED(user_vec, items_vecs)
    topn_iids = np.argsort(dists, axis=1)[:, :topn]
    topn_iids_item = [iid_to_item_id[iid] for iid in topn_iids.reshape(-1)]
    return topn_iids_item

In [ ]:
recos = {}
users = list(user_id_to_uid.keys())
for user_id in tqdm(users):
    recos_for_user = recommend_to_user(user_id, items_vecs)
    recos.update({user_id: recos_for_user})

  0%|          | 0/65974 [00:00<?, ?it/s]

In [ ]:
recos[699317]

[16270, 2722, 3182, 10761, 12743, 5411, 7571, 16166, 11310, 3734]

In [ ]:
recos_df = pd.DataFrame(data=recos)

In [ ]:
recos_df = recos_df.T

In [ ]:
recos_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65974 entries, 176549 to 30545
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       65974 non-null  int64
 1   1       65974 non-null  int64
 2   2       65974 non-null  int64
 3   3       65974 non-null  int64
 4   4       65974 non-null  int64
 5   5       65974 non-null  int64
 6   6       65974 non-null  int64
 7   7       65974 non-null  int64
 8   8       65974 non-null  int64
 9   9       65974 non-null  int64
dtypes: int64(10)
memory usage: 7.6 MB


In [ ]:
recos_df.to_excel('recos_dssm.xlsx')

In [ ]:
data = pd.read_excel("/content/reco_dssm.xlsx", names = ['user_id', 'item_id'], index_col=None)

In [ ]:
data

,user_id,item_id
0,21,"7102, 4457, 8636, 1695, 14961, 5505, 11085, 78..."
1,53,"7102, 4457, 3734, 8636, 4495, 12915, 4151, 144..."
2,60,"3734, 11237, 5434, 142, 2720, 5658, 4151, 1291..."
3,81,"8636, 14607, 7102, 7417, 12501, 4495, 5266, 14..."
4,93,"3734, 8636, 11118, 7102, 12915, 4151, 142, 272..."
...,...,...
65968,1097486,"7102, 9214, 4457, 3734, 14431, 8636, 5266, 145..."
65969,1097489,"13913, 565, 9506, 4643, 9499, 9828, 741, 10565..."
65970,1097508,"11664, 16270, 14942, 4475, 973, 2722, 6774, 56..."
65971,1097513,"14961, 1695, 5505, 10903, 13446, 9728, 7899, 1..."


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65973 entries, 0 to 65972
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  65973 non-null  int64 
 1   item_id  65973 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [ ]:
data.to_csv('recos_dssm.csv', sep = ';')

In [ ]:
dssm_recos = pd.read_excel('/content/reco_dssm1.xlsx')

dssm_recos

,user_id,item_id
0,176549,"9728, 13865, 7626, 12173, 1785, 11310, 10440, ..."
1,699317,"13545, 7571, 3182, 16166, 13243, 16270, 7582, ..."
2,1016458,"12356, 1418, 7829, 13849, 6162, 9728, 13865, 1..."
3,896751,"3606, 15646, 14703, 12366, 9159, 6626, 8447, 1..."
4,141674,"7571, 11756, 16166, 6033, 3182, 5411, 16270, 1..."
...,...,...
13349,231645,"13865, 10440, 4880, 9728, 15297, 4151, 3734, 1..."
13350,13647,"9728, 10440, 13865, 142, 15297, 4151, 1844, 37..."
13351,43990,"7571, 9728, 13018, 13865, 3734, 16166, 3182, 1..."
13352,942675,"10440, 9728, 15297, 13865, 4151, 3734, 4880, 1..."


In [ ]:
users = dssm_recos['user_id'].unique()
dssm_recos['item_id'] = dssm_recos['item_id'].apply(lambda x: list(map(int, x.split(", "))))


def dssm_offline_reco(user_id):
    if user_id in users:
        user_recos = dssm_recos[dssm_recos['user_id'] == user_id]['item_id'].to_list()[0]
        return user_recos

In [ ]:
dssm_offline_reco(1097513)

[9728, 11754, 13865, 5693, 1785, 15464, 1287, 10440, 12995, 4436]